#### Projet Fouille de texte : Classification thématique de chapôs du Monde 

> Ji AN

Ce petit script est simplement pour but de tester les différents modèles de la librairie scikit-learn et d'observer leurs performances.

Modèles à importer :

- ComplementNB
- LogisticRegression
- DecisionTreeClassifier
- VotingClassifier  



In [2]:
from spacy_tokenizer import spacy_tokenizer
from sklearn import datasets, metrics
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
chapos = datasets.load_files('./Corpus', encoding='utf-8', shuffle=True)
print(chapos.data[:5])

['La section parallèle centrée sur le cinéma d’auteur et de découverte\xa0a également attribué le Prix du jury à «\xa0Joyland\xa0», premier long-métrage du Pakistanais Saim Sadiq.', 'La réalisatrice Patricia Mazuy explore l’origine de la pulsion meurtrière dans une tragédie qui voit deux frères se déchirer autour de l’image du père défunt.', 'La grand-messe du cinéma français qui se tient vendredi célèbre l’actrice australienne, récompensée pour l’ensemble de sa carrière.', 'En\xa02002, la cinéaste Marie-Claude Treilhou réalisait ce film splendide sur un fait divers qui sème la zizanie entre quatre amies. Il ressort en salle en copie restaurée.', 'Pendant qu’Emmanuel Macron était en tournée diplomatique aux sommets du G7 et de l’OTAN, les députés ont élu les membres des instances du Palais-Bourbon.']


In [4]:
for t in chapos.target[:5]:
    print(chapos.target_names[t])

cinema
cinema
cinema
cinema
politqiue


In [5]:
# We first remove the words that appear in less than 2 documents or in more than 90% of the documents
# We then choose the 1000 most frequent words as features
# We finally apply our custom tokenizer using spaCy (see spacy_tokenizer.py) in a simple way
vectorizer = CountVectorizer(min_df=2, max_df=0.9, max_features=1000, tokenizer=spacy_tokenizer, token_pattern=None)
X = vectorizer.fit_transform(chapos.data)
y = chapos.target
print(f"The corpus has {X.shape[0]} instances and {X.shape[1]} attributes")
print(f"The corpus has {len(chapos.target_names)} classes: {chapos.target_names}")

The corpus has 478 instances and 1000 attributes
The corpus has 2 classes: ['cinema', 'politqiue']


In [6]:
# We do a simple train/test split (80/20) with a random state of 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# call the 4 classifiers
clf1 = ComplementNB()
clf2 = LogisticRegression(random_state=1)
clf3 = DecisionTreeClassifier(random_state=1)
eclf = VotingClassifier(estimators=[('cnb', clf1), ('lr', clf2), ('dt', clf3)], voting='hard')

In [10]:
# loop over the classifiers, do the prediction, print their classification report
for clf in (clf1, clf2, clf3, eclf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f"\n==== {clf.__class__.__name__} ====\n")
    print(metrics.classification_report(y_test, y_pred, target_names=chapos.target_names, digits=3))


==== ComplementNB ====

              precision    recall  f1-score   support

      cinema      0.939     0.958     0.948        48
   politqiue      0.957     0.938     0.947        48

    accuracy                          0.948        96
   macro avg      0.948     0.948     0.948        96
weighted avg      0.948     0.948     0.948        96


==== LogisticRegression ====

              precision    recall  f1-score   support

      cinema      0.852     0.958     0.902        48
   politqiue      0.952     0.833     0.889        48

    accuracy                          0.896        96
   macro avg      0.902     0.896     0.895        96
weighted avg      0.902     0.896     0.895        96


==== DecisionTreeClassifier ====

              precision    recall  f1-score   support

      cinema      0.864     0.792     0.826        48
   politqiue      0.808     0.875     0.840        48

    accuracy                          0.833        96
   macro avg      0.836     0.833    